# Martech - Load Data

## Initialise Connection

In [1]:
from astra_db import AstraSession
from cassandra import ConsistencyLevel

session = AstraSession().session

## Define Prepared Statements

### Communications Table

In [2]:
# Communications Table

# communication_id: UUID.
# comm_date: Timestamp (rounded to the nearest hour).

insert_communication_stmt = session.prepare("INSERT INTO communications (communication_id, comm_date, customer_id, channel, category_group, category, activity_name) VALUES (?, ?, ?, ?, ?, ?, ?)")
insert_communication_stmt.consistency_level = ConsistencyLevel.ALL

### PROFILECAP

In [3]:
# PROFILECAP

update_profile_cap_stmt = session.prepare("UPDATE profile_cap SET count = count + 1 WHERE customer_id = ? AND channel = ? AND comm_date_bucket = ?")
update_profile_cap_stmt.consistency_level = ConsistencyLevel.ALL

### ACTYCAP

In [4]:
# ACTYCAP

update_acty_cap_stmt = session.prepare("UPDATE acty_cap SET count = count + 1 WHERE activity_name = ? AND comm_date_bucket = ?")
update_acty_cap_stmt.consistency_level = ConsistencyLevel.ALL

### PRTYCAP

In [5]:
# PRTYCAP

update_prty_cap_stmt = session.prepare("UPDATE prty_cap SET count = count + 1 WHERE category_group = ? AND category = ? AND comm_date_bucket = ?")
update_prty_cap_stmt.consistency_level = ConsistencyLevel.ALL

### CHANNELCAP

In [6]:
# CHANNELCAP

update_channel_cap_stmt = session.prepare("UPDATE channel_cap SET count = count + 1 WHERE channel = ? AND comm_date_bucket = ?")
update_channel_cap_stmt.consistency_level = ConsistencyLevel.ALL

## Load Data

### Define Entities

In [7]:
# Define Entities

# customer_id
customer_id_array = [
    '0b6ec30f-8b1a-406f-82f6-89222a017449',
    '8435a9b9-9e88-495c-a297-dfd25d44428b',
    '8e12d570-e47d-43b9-9435-b2020899860b',
    '9e53cd14-0316-46ff-b913-30228fcc5b73',
    'a6e198e0-674d-4a27-ba01-595a78fda2ef',
    'af9e65bd-4474-40c9-8d16-7b82035d3e0f',
    'b191cef3-4624-4e88-9a93-73de506e4be7',
    'bcabe5dd-8675-4a88-94ed-a9faa9473355',
    'cf3a5d68-e66f-4ee7-a2d1-2666c5366540',
    'f68e8bc9-86b7-4c8e-ba74-87d31d27a531',
    'bcabe5dd-8675-4a88-94ed-a9faa9473355',
]

# channel
channel_array = ['email', 'sms', 'push', 'in-app', 'direct']

# category_group
category_group_array = ['ctgy_grp_1', 'ctgy_grp_2', 'ctgy_grp_3', 'ctgy_grp_4', 'ctgy_grp_5']

# category
category_array = ['ctgy_1', 'ctgy_2', 'ctgy_3', 'ctgy_4', 'ctgy_5']

# activity_name
activity_name_array = ['credit_card', 'mortgage', 'business', 'retail', 'investment']

### Insert Data

In [23]:
from datetime import datetime
import random
import uuid

# iterate over month
for day in range(1, 31):
    print(day)

    # iterate over hours in a day
    for hour in range(0, 24):
        print(f"{hour} ", end='')

        # iterate over minutes in an hour
        for minute in range(0, 60):
            #print(f"{minute} ", end='')

            date = datetime(2024, 12, day, hour, minute, 00)
            comm_date = date.strftime('%Y-%m-%d %H:%M:%S')
            comm_date_bucket = date.strftime('%Y%m%d%H') # bucket by hour
            #print(f"{comm_date} - {comm_date_bucket}")
            
            # generate values
            comm_id = str(uuid.uuid4())
            customer_id = customer_id_array[random.randint(0, 9)]
            channel = channel_array[random.randint(0, 4)]
            category_group = category_group_array[random.randint(0, 4)]
            category = category_array[random.randint(0, 4)]
            activity_name = activity_name_array[random.randint(0, 4)]

            # insert communication
            session.execute(
                insert_communication_stmt, 
                (comm_id, date, customer_id, channel, category_group, category, activity_name)
            )


            # Update Profile Cap
            session.execute(
                update_profile_cap_stmt, 
                (customer_id, channel, comm_date_bucket)
            )

            # Update Acty Cap
            session.execute(
                update_acty_cap_stmt, 
                (activity_name, comm_date_bucket)
            )

            # Update Prty Cap
            session.execute(
                update_prty_cap_stmt, 
                (category_group, category, comm_date_bucket)
            )

            # Update Channel Cap
            session.execute(
                update_channel_cap_stmt, 
                (channel, comm_date_bucket)
            )

1
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 2
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 3
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 4
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 5
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 6
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 7
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 8
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 9
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 10
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 11
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 12
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 13
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 14
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 15
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 16
0 1 2 3 4 5 6 7 8 9 10 11 12 13